<h1>Ensaio para Active Voxel</h1>

In [6]:
from vpython import *

ctr = vec( 150, 50, 150 )

# This is needed in Jupyter notebook and lab to make programs easily rerunnable
scene = canvas( center=ctr, background = color.gray( 0.8 ))
scene.autoscaling = False
scene.forward = vec( 0, 200, 0 )
scene.up = vec( 0, 0, 1 )
scene.camera.pos = vec( 150, -330, 150 )
scene.range = 50

<IPython.core.display.Javascript object>

In [13]:
# quads always start with 6 quads needed to form the seed
# <(( int e1, int e2 ) t1, ( int e3, int e4 ) t2 ), int >
quads = {
    # front
    (( 1, 2 ), ( 3, 4 )): 0,

    # top
    (( -3, 5 ), ( 6, 7 )): 1,

    # back
    (( -6, 8 ), ( 9, 10 )): 2,

    # bottom
    (( -9, 11 ), ( -1, 12 )): 3,

    # left
    (( -11, -8 ), ( -5, -2 )): 4,

    # right
    (( -12, -4 ), ( -7, -10 )): 5
}

print( quads )

{((1, 2), (3, 4)): 0, ((-3, 5), (6, 7)): 1, ((-6, 8), (9, 10)): 2, ((-9, 11), (-1, 12)): 3, ((-11, -8), (-5, -2)): 4, ((-12, -4), (-7, -10)): 5}


In [16]:
quads_rev = {}

# dict comprehension failed here
for quad in quads:
    quads_rev[ quads[ quad ]] = quad
    
print( quads_rev )

{0: ((1, 2), (3, 4)), 1: ((-3, 5), (6, 7)), 2: ((-6, 8), (9, 10)), 3: ((-9, 11), (-1, 12)), 4: ((-11, -8), (-5, -2)), 5: ((-12, -4), (-7, -10))}


In [19]:
edges_quads = {
    1: ( 0, 3 ),  2: ( 0, 4 ),  3: ( 0, 1 ),  4: ( 0, 5 ),
    5: ( 1, 4 ),  6: ( 1, 2 ),  7: ( 1, 5 ),  8: ( 2, 4 ),
    9: ( 2, 3 ), 10: ( 2, 5 ), 11: ( 3, 4 ), 12: ( 3, 5 )
}

print( edges_quads )

{1: (0, 3), 2: (0, 4), 3: (0, 1), 4: (0, 5), 5: (1, 4), 6: (1, 2), 7: (1, 5), 8: (2, 4), 9: (2, 3), 10: (2, 5), 11: (3, 4), 12: (3, 5)}


In [24]:
scale = 1.0

vertices = {
    # front
    # edge 1 = vx 1 -> 2
    1: ( scale, 0, 0, False ),     # vx 1
    2: ( 0, 0, 0, False ),         # vx 2

    # edge 2 = vx 2 (shared edge 1) -> 3
    3: ( 0, scale, 0, False ),     # vx 3

    # edge 3 = vx 3 (shared edge 2) -> 4
    4: ( scale, scale, 0, False ), # vx 4

    # edge 4 = vx 4 (shared edge 3) -> vx1 (shared edge 1) 

    # top
    # -1 * edge 3

    # edge 5 = vx 3 (shared edge 2) -> 5
    5: ( 0, scale, scale, False ), # vx 5

    # edge 6 = vx 5 (shared edge 5) -> 6
    6: ( scale, scale, scale, False ),# vx 6

    # edge 7 = vx 6 (shared edge 6) -> vx 4 (shared edge 3)

    # back
    # -1 * edge 6

    # edge 8 = vx 5 (shared edge 5) -> 7
    7: ( 0, 0, scale, False ),     # vx 7

    # edge 9 = vx 7 (shared edge 8) -> 8
    8: ( scale, 0, scale, False ) # vx 8

    # edge 10 = vx 8 (shared edge 9) -> vx 6 (shared edge 6)

    # bottom
    # -1 * edge 9
    # edge 11 = vx 7 (shared edge 8 ) -> vx 2 (shared edge 1 )
    # -1 * edge 1
    # edge 12 = vx 1 (shared edge 1 ) -> vx 8 (shared edge 9 )

    # left
    # -1 * edge 11, -1 * edge 8, -1 * edge 5, -1 * edge 2

    # right
    # -1 * edge 12, -1 * edge 4, -1 * edge 7, -1 * edge 10
}

print( vertices )

{1: (1.0, 0, 0, False), 2: (0, 0, 0, False), 3: (0, 1.0, 0, False), 4: (1.0, 1.0, 0, False), 5: (0, 1.0, 1.0, False), 6: (1.0, 1.0, 1.0, False), 7: (0, 0, 1.0, False), 8: (1.0, 0, 1.0, False)}


In [26]:
# < int, List<( int edge, int t, int pos )>>

parent_edges = {
    1: [{ "edge": 1, "t": 1, "pos": 0 }, { "edge":  4, "t": 1, "pos": 1 }, { "edge": 12, "t": 1, "pos": 0 }],
    2: [{ "edge": 1, "t": 1, "pos": 1 }, { "edge":  2, "t": 1, "pos": 0 }, { "edge": 11, "t": 1, "pos": 1 }],
    3: [{ "edge": 2, "t": 1, "pos": 1 }, { "edge":  3, "t": 1, "pos": 0 }, { "edge":  5, "t": 1, "pos": 0 }],
    4: [{ "edge": 3, "t": 1, "pos": 1 }, { "edge":  4, "t": 1, "pos": 0 }, { "edge":  7, "t": 1, "pos": 1 }],
    5: [{ "edge": 5, "t": 1, "pos": 1 }, { "edge":  6, "t": 1, "pos": 0 }, { "edge":  8, "t": 1, "pos": 0 }],
    6: [{ "edge": 6, "t": 1, "pos": 1 }, { "edge":  7, "t": 1, "pos": 0 }, { "edge": 10, "t": 1, "pos": 1 }],
    7: [{ "edge": 8, "t": 1, "pos": 1 }, { "edge":  9, "t": 1, "pos": 0 }, { "edge": 11, "t": 1, "pos": 0 }],
    8: [{ "edge": 9, "t": 1, "pos": 1 }, { "edge": 10, "t": 1, "pos": 0 }, { "edge": 12, "t": 1, "pos": 1 }]
}

print( parent_edges )

{1: [{'edge': 1, 't': 1, 'pos': 0}, {'edge': 4, 't': 1, 'pos': 1}, {'edge': 12, 't': 1, 'pos': 0}], 2: [{'edge': 1, 't': 1, 'pos': 1}, {'edge': 2, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 1}], 3: [{'edge': 2, 't': 1, 'pos': 1}, {'edge': 3, 't': 1, 'pos': 0}, {'edge': 5, 't': 1, 'pos': 0}], 4: [{'edge': 3, 't': 1, 'pos': 1}, {'edge': 4, 't': 1, 'pos': 0}, {'edge': 7, 't': 1, 'pos': 1}], 5: [{'edge': 5, 't': 1, 'pos': 1}, {'edge': 6, 't': 1, 'pos': 0}, {'edge': 8, 't': 1, 'pos': 0}], 6: [{'edge': 6, 't': 1, 'pos': 1}, {'edge': 7, 't': 1, 'pos': 0}, {'edge': 10, 't': 1, 'pos': 1}], 7: [{'edge': 8, 't': 1, 'pos': 1}, {'edge': 9, 't': 1, 'pos': 0}, {'edge': 11, 't': 1, 'pos': 0}], 8: [{'edge': 9, 't': 1, 'pos': 1}, {'edge': 10, 't': 1, 'pos': 0}, {'edge': 12, 't': 1, 'pos': 1}]}


In [31]:
# <( int edge, int t, int pos ), int >

vertex_in_edge = {}

for vx in parent_edges:
    for edge in parent_edges[ vx ]:
        vertex_in_edge[ str( edge )] = vx
    
print( vertex_in_edge )


{"{'edge': 1, 't': 1, 'pos': 0}": 1, "{'edge': 4, 't': 1, 'pos': 1}": 1, "{'edge': 12, 't': 1, 'pos': 0}": 1, "{'edge': 1, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 0}": 2, "{'edge': 11, 't': 1, 'pos': 1}": 2, "{'edge': 2, 't': 1, 'pos': 1}": 3, "{'edge': 3, 't': 1, 'pos': 0}": 3, "{'edge': 5, 't': 1, 'pos': 0}": 3, "{'edge': 3, 't': 1, 'pos': 1}": 4, "{'edge': 4, 't': 1, 'pos': 0}": 4, "{'edge': 7, 't': 1, 'pos': 1}": 4, "{'edge': 5, 't': 1, 'pos': 1}": 5, "{'edge': 6, 't': 1, 'pos': 0}": 5, "{'edge': 8, 't': 1, 'pos': 0}": 5, "{'edge': 6, 't': 1, 'pos': 1}": 6, "{'edge': 7, 't': 1, 'pos': 0}": 6, "{'edge': 10, 't': 1, 'pos': 1}": 6, "{'edge': 8, 't': 1, 'pos': 1}": 7, "{'edge': 9, 't': 1, 'pos': 0}": 7, "{'edge': 11, 't': 1, 'pos': 0}": 7, "{'edge': 9, 't': 1, 'pos': 1}": 8, "{'edge': 10, 't': 1, 'pos': 0}": 8, "{'edge': 12, 't': 1, 'pos': 1}": 8}


In [39]:
def quads_vertices():
    res = []
    
    for vx in vertices:
        res.append([ vertices[ vx ][ 0 ], vertices[ vx ][ 1 ], vertices[ vx ][ 2 ]])
    
    return res

mesh_vertices = quads_vertices()

print( mesh_vertices )

[[1.0, 0, 0], [0, 0, 0], [0, 1.0, 0], [1.0, 1.0, 0], [0, 1.0, 1.0], [1.0, 1.0, 1.0], [0, 0, 1.0], [1.0, 0, 1.0]]


In [ ]:
def edges_GetTri( edge1, edge2 ):
    vxs = []

    # quads keep edges on a one-based array because negative indicates
    # that we have to consider the vertices in reverse order in that edge.

    id = edge1 if edge1 > 0 else -1 * edge1
    vxs[ 0 ] = edge1 > 0 ? edges[id].start : edges[id].end;
    vxs[ 1 ] = edge1 > 0 ? edges[id].end : edges[id].start;

    id = edge2 > 0 ? edge2 : -1 * edge2;
    vxs[2] = edge2 > 0 ? edges[id].start : edges[id].end;
    vxs[3] = edge2 > 0 ? edges[id].end : edges[id].start;

    # edges must share a vertex and be informed in correct order
    if (vxs[1] != vxs[2])
        return new int[] { };
    else
        # one based to zero based
        return new int[] { vxs[0] - 1, vxs[1] - 1, vxs[3] - 1 };


def quads_GetTris()
{
    res = []
    for i in range( 0, len( quads )):
        quad = quads_rev[ i ]

        res.append( edges_GetTri( quad.t1.e1, quad.t1.e2 ))
        res.append( edges_GetTri( quad.t2.e3, quad.t2.e4 ))

    return res #.SelectMany( t => new[] { t }).ToArray().SelectMany(x => x).ToArray();

    
print( quads_GetTris())
